In [5]:
"""
-Arman Bazarchi-
Temporal_Landing
...



"""

from datasets import load_dataset
from minio import Minio
from PIL import Image
from tqdm import tqdm
import io
import pandas as pd
import os
import time
import requests
import re

# ==============================
# 1️⃣ Configuration
# ==============================
MINIO_ENDPOINT = "localhost:9000"
ACCESS_KEY = "admin"
SECRET_KEY = "password123"

ROOT_BUCKET = "landing-zone"         # Top-level bucket
TEMP_PREFIX = "temporal-landing"     # Subfolder prefix inside landing-zone

# Metadata config
METADATA_REMOTE_PATH = f"{TEMP_PREFIX}/metadata/metadata_final.csv"
METADATA_LOCAL = "metadata_final.csv"

# Limits
MAX_PER_SPECIES = 20
MAX_SPECIES_PER_FAMILY = 5
MAX_TOTAL_IMAGES = 100000

# Known snake families
SNAKE_FAMILIES = {
    "Viperidae", "Elapidae", "Colubridae", "Pythonidae", "Boidae",
    "Typhlopidae", "Lamprophiidae", "Natricidae", "Dipsadidae",
    "Leptotyphlopidae", "Xenopeltidae", "Anomalepididae", "Loxocemidae",
    "Uropeltidae", "Cylindrophiidae", "Aniliidae", "Acrochhordidae",
    "Anomochilidae", "Atractaspididae", "Bolyeridae"
}

# ==============================
# 2️⃣ Initialize MinIO client
# ==============================
client = Minio(
    MINIO_ENDPOINT,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure=False
)

# ==============================
# 3️⃣ Ensure landing-zone and temporal-landing exist
# ==============================
if not client.bucket_exists(ROOT_BUCKET):
    client.make_bucket(ROOT_BUCKET)
    print(f"✅ Created top-level bucket: {ROOT_BUCKET}")

# Create temporal-landing subfolder if missing
temporal_exists = any(
    obj.object_name.startswith(f"{TEMP_PREFIX}/")
    for obj in client.list_objects(ROOT_BUCKET, recursive=False)
)
if not temporal_exists:
    client.put_object(
        ROOT_BUCKET,
        f"{TEMP_PREFIX}/.init",
        data=io.BytesIO(b"init"),
        length=4,
        content_type="text/plain"
    )
    print(f"✅ Created subfolder: {TEMP_PREFIX}/ inside {ROOT_BUCKET}")

# ==============================
# 4️⃣ Check for existing metadata
# ==============================
try:
    client.fget_object(ROOT_BUCKET, METADATA_REMOTE_PATH, METADATA_LOCAL)
    metadata_df = pd.read_csv(METADATA_LOCAL)
    print(f"📂 Loaded metadata from {TEMP_PREFIX} with {len(metadata_df)} records.")

except Exception:
    metadata_df = pd.DataFrame(columns=[
        "uuid","temporal_path", "kingdom", "phylum", "class", "order", "family",
        "genus", "species", "scientific_name", "common", "image_url"
    ])
    print("🆕 No metadata found in temporal-landing — starting fresh.")
# Maintain fast lookup sets
existing_uuids = set(metadata_df["uuid"].dropna().unique()) if "uuid" in metadata_df.columns else set()

# ==============================
# 5️⃣ Scan existing images
# ==============================
print("🔍 Scanning existing images in temporal-landing...")

existing_image_ids = set()
for obj in client.list_objects(ROOT_BUCKET, prefix=f"{TEMP_PREFIX}/images/", recursive=True):
    # Extract UUID from filename: temporal-landing/images/<uuid>.jpg
    match = re.match(rf"{TEMP_PREFIX}/images/([a-f0-9\-]+)\.jpg", obj.object_name, re.IGNORECASE)
    if match:
        existing_image_ids.add(match.group(1))

print(f"📸 Found {len(existing_image_ids)} existing image files in temporal-landing.")

family_species = {}
species_counts = {}

# ==============================
# 6️⃣ Build family→species map from metadata
# ==============================
if not metadata_df.empty:
    for _, row in metadata_df.iterrows():
        fam = row.get("family")
        sp = row.get("species")
        if pd.notna(fam) and pd.notna(sp):
            family_species.setdefault(fam, set()).add(sp)
            species_counts[sp] = species_counts.get(sp, 0) + 1

print(f"🌿 Tracking {len(family_species)} families from existing metadata.")

# ==============================
# 7️⃣ Progressive Chunk Loop
# ==============================
for chunk_idx in range(0, 5):
    print(f"\n🔹 Processing chunk {chunk_idx}%–{chunk_idx+1}% ...")

    try:
        dataset = load_dataset(
            "imageomics/TreeOfLife-200M",
            split=f"train[{chunk_idx}%:{chunk_idx+1}%]"
        )
    except Exception as e:
        print(f"⚠️ Skipping chunk {chunk_idx}: {e}")
        continue

    for sample in tqdm(dataset):
        species = sample.get("species") or "unknown"
        kingdom = sample.get("kingdom")
        cls = sample.get("class")
        order = sample.get("order")
        family = sample.get("family") or "unknown"
        img_id = sample.get("uuid")
        image_url = sample.get("source_url")

        # Only save snake images
        if not (
            kingdom and kingdom.strip().lower() == "animalia"
            and cls and cls.strip().lower() == "squamata"
            and family in SNAKE_FAMILIES
        ):
            continue

        count = species_counts.get(species, 0)
        if count >= MAX_PER_SPECIES:
            continue

        current_species_in_family = family_species.get(family, set())
        if len(current_species_in_family) >= MAX_SPECIES_PER_FAMILY and species not in current_species_in_family:
            continue

        try:
            if img_id not in existing_image_ids:
                response = requests.get(image_url, timeout=10)
                response.raise_for_status()
                image_bytes = response.content

                img = Image.open(io.BytesIO(image_bytes))
                img.verify()
                buffer = io.BytesIO(image_bytes)

                # Save to MinIO (include TEMP_PREFIX)
                object_name = f"{TEMP_PREFIX}/images/{img_id}.jpg"
                client.put_object(
                    ROOT_BUCKET,
                    object_name,
                    data=buffer,
                    length=len(image_bytes),
                    content_type="image/jpeg"
                )
                existing_image_ids.add(img_id)

            if img_id not in existing_uuids:
                # Save metadata
                record = {
                    "uuid": img_id,
                    "temporal_path": object_name,
                    "kingdom": kingdom,
                    "phylum": sample.get("phylum"),
                    "class": cls,
                    "order": order,
                    "family": family,
                    "genus": sample.get("genus"),
                    "species": species,
                    "scientific_name": sample.get("scientific_name"),
                    "common": sample.get("common"),
                    "image_url": image_url
                }

                metadata_df = pd.concat([metadata_df, pd.DataFrame([record])], ignore_index=True)
                metadata_df.to_csv(METADATA_LOCAL, index=False)
                client.fput_object(ROOT_BUCKET, METADATA_REMOTE_PATH, METADATA_LOCAL, content_type="text/csv")

                species_counts[species] = count + 1
                family_species.setdefault(family, set()).add(species)
                existing_uuids.add(img_id)

                print(f"✅ Saved {species} ({species_counts[species]}/{MAX_PER_SPECIES}) "
                      f"in family {family} ({len(family_species.get(family, []))}/{MAX_SPECIES_PER_FAMILY})")

        except Exception as e:
            print(f"⚠️ Error saving {species}: {e}")
            continue

        if len(metadata_df) >= MAX_TOTAL_IMAGES:
            print("🛑 Reached global image limit.")
            break

    if len(metadata_df) >= MAX_TOTAL_IMAGES:
        break

    time.sleep(1)

print(f"✅ Finished progressive sampling. Total images: {len(metadata_df)}")

# ==============================
# 8️⃣ Cleanup local CSV
# ==============================
if os.path.exists(METADATA_LOCAL):
    os.remove(METADATA_LOCAL)
    print(f"🗑️ Removed local metadata file: {METADATA_LOCAL}")


✅ Created subfolder: temporal-landing/ inside landing-zone
🆕 No metadata found in temporal-landing — starting fresh.
🔍 Scanning existing images in temporal-landing...
📸 Found 0 existing image files in temporal-landing.
🌿 Tracking 0 families from existing metadata.

🔹 Processing chunk 0%–1% ...


  0%|                                                                          | 16/2139373 [00:01<73:46:05,  8.06it/s]

✅ Saved constrictor (1/20) in family Colubridae (1/5)


  0%|                                                                          | 97/2139373 [00:03<20:42:51, 28.69it/s]

✅ Saved sipedon (1/20) in family Colubridae (2/5)


...

✅ Saved totonacus (7/20) in family Viperidae (5/5)


 99%|█████████████████████████████████████████████████████████████████████▌| 2124448/2139373 [09:43<00:10, 1457.76it/s]

✅ Saved bicolor (10/20) in family Loxocemidae (1/5)


100%|██████████████████████████████████████████████████████████████████████| 2139373/2139373 [09:46<00:00, 3647.99it/s]


✅ Finished progressive sampling. Total images: 736
🗑️ Removed local metadata file: metadata_final.csv
